# 「tflite micro」であそぼう！
## 元ノートブック：[@dansitu](https://twitter.com/dansitu)
### 日本語バーション：[@proppy](https://twitter.com/proppy])


# github.com/proppy/TfLiteMicroArduino ← PRをどうぞ


# 「tflite micro」ってなんだ？

- マイコンで「tflite」が動く事
![img](https://media.digikey.com/Photos/Espressif%20Systems/ESP32-WROOM-32.JPG)
- https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/experimental/micro

In [ ]:
! python -m pip install tensorflow==2.0.0-beta1
import tensorflow as tf
print(tf.version.VERSION)

! python -m pip install matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

# 一番かんたんなモデルを作りましょう！

## sin() 1000個

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

x_values = np.random.uniform(low=0, high=2*math.pi, size=1000)
np.random.shuffle(x_values)
y_values = np.sin(x_values)
plt.plot(x_values, y_values, 'b.')
print(plt.show())

## ノイズをかけて

In [ ]:
y_values += 0.1 * np.random.randn(*y_values.shape)
plt.plot(x_values, y_values, 'b.')
plt.show()

## datasetをちゃんと分けて

In [ ]:
x_train, x_test, x_validate = x_values[:600], x_values[600:800], x_values[800:]
y_train, y_test, y_validate = y_values[:600], y_values[600:800], y_values[800:]

plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.plot(x_validate, y_validate, 'y.', label="Validate")
plt.legend()
plt.show()

## Kerasで２分を温めて

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(x_train, y_train, epochs=1000, batch_size=16,
                    validation_data=(x_validate, y_validate), verbose=1)

## モデルを試して

In [ ]:
predictions = model.predict(x_test)

plt.clf()
plt.plot(x_test, y_test, 'bo', label='Test')
plt.plot(x_test, predictions, 'ro', label='Keras')
plt.legend()
plt.show()

## tfliteにゆっくり変わって

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("sine_model_data.tflite", "wb").write(tflite_model)

## マイコンに入れる前に最後の確認

In [ ]:
interpreter = tf.lite.Interpreter('sine_model_data.tflite')
interpreter.allocate_tensors()
input = interpreter.tensor(interpreter.get_input_details()[0]["index"])
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
lite_predictions = np.empty(x_test.size)
for i in range(x_test.size):
    input()[0] = x_test[i]
    interpreter.invoke()
    lite_predictions[i] = output()[0]
plt.plot(x_test, y_test, 'bo', label='Test')
plt.plot(x_test, predictions, 'ro', label='Keras')
plt.plot(x_test, lite_predictions, 'kx', label='TFLite')
plt.legend()
plt.show()

## マイコンに入れるために「ANSI C」に変わって

In [ ]:
_ = ! which xxd || apt-get install xxd
! xxd -i sine_model_data.tflite > sine_model_data.h
try:
    from google.colab import files
    files.download('sine_model_data.h')
except Exception as e:
    from IPython.display import FileLink
    display(FileLink('sine_model_data.h'))

```
unsigned char sine_model_data_tflite[][] = {
  0x18, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x0e, 0x00,
  0x18, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x0e, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x10, 0x0a, 0x00, 0x00,
  0xb8, 0x05, 0x00, 0x00, 0xa0, 0x05, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x0b, 0x00, 0x00, 0x00, 0x90, 0x05, 0x00, 0x00, 0x7c, 0x05, 0x00, 0x00,
  0x24, 0x05, 0x00, 0x00, 0xd4, 0x04, 0x00, 0x00, 0xc4, 0x00, 0x00, 0x00,
  // ...
}
unsigned int sine_model_data_tflite_len = 2640;
```

## TFLiteMicroのやり方

```
#include "tensorflow/lite/experimental/micro/kernels/all_ops_resolver.h"
#include "tensorflow/lite/experimental/micro/micro_error_reporter.h"
#include "tensorflow/lite/experimental/micro/micro_interpreter.h"
#include "tensorflow/lite/schema/schema_generated.h"
#include "tensorflow/lite/version.h"

int main() {
  // ...
}
```

## Modelをget
```
// ...
#include "tensorflow/lite/experimental/micro/examples/hello_world/sine_model_data.h"

int main() {
  // ...
  const tflite::Model* model = tflite::GetModel(g_sine_model_data);
}
```

## MicroなんとかのInit

```
  uint8_t tensor_arena[2 * 1024];
  static tflite::MicroErrorReporter micro_error_reporter;
  tflite::ops::micro::AllOpsResolver resolver;

  tflite::MicroInterpreter interpreter(
      model, resolver, tensor_arena, sizeof(tensor_arena), error_reporter);
  TfLiteStatus allocate_status = interpreter.AllocateTensors();
```

## TFLiteそのまま

```
  TfLiteTensor* input = interpreter.input(0);
  TfLiteTensor* output = interpreter.output(0);

  input.data.f[0] = x_val;
  TfLiteStatus invoke_status = interpreter.Invoke();
  float y_val = output.data.f[0];
```

## Arduino形

```
//　#include　"何か.h"

// グローバル

void setup() {
    // 設定
}

void loop() {
    // 毎回
}

```

## Arduinoの匂いがする！

```
#include "TfLiteMicroArduino.h"
#include "sine_model_data.h"

void setup() {
  TfLiteMicro.begin(sine_model_data_tflite);
}

void loop() {
  // ...
  TfLiteMicro.inputFloat(0)[0] = x_val;
  TfLiteMicro.invoke();
  float y_val = TfLiteMicro.outputFloat(0)[0]);
}
```
